<a href="https://colab.research.google.com/github/siddhanthole74/Fake-News-Prediction-using-Machine-Learning/blob/main/Project5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1.id: unique id for a news article

2.title: the title of a news article

3.author: author of the news article

4.text: the text of the article; could be incomplete

5.label: a label that marks whether the news article is real or fake:

    1: Fake news
    0: real News

in preprocessing algorithum basically does not understand the text it only understand the number soo we have to convert it basically into corresponding meaningful number

Binary classification is basically done using Logistic regression in this project that is news is basically real or fake

Importing the dependencies 

In [22]:
import pandas as pd
import numpy as np
import re # re stands for regular expression it is very useful for searching the text in the document 
from nltk.corpus import stopwords #nltk stands for natural language toolkit. corpus generally means body of that particular text . the important content of that text stopwords are basically tjose words that are not useful for our paragraph 
from nltk.stem.porter import PorterStemmer # we basically we perform the function called steamming . steamming basicaslly takes the word and remove prefix and suffix from the word . and return the rootword of it
from sklearn.feature_extraction.text import TfidfVectorizer # soo basically we TfidfVectorizor are used to convert text into feature vector soo basically feature vector in basically numbers  
from sklearn.model_selection import train_test_split # to split the model into train test 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [23]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
print(stopwords.words('english')) # soo these words doesnot add much values to our dataset 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing 

In [25]:
news_dataset=pd.read_csv('/content/train.csv',error_bad_lines=False, engine="python")


<ipython-input-25-98a311de4f42>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  news_dataset=pd.read_csv('/content/train.csv',error_bad_lines=False, engine="python")
Skipping line 1259: unexpected end of data


In [26]:
news_dataset.shape

(1257, 5)

In [27]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
news_dataset.isnull().sum()

id          0
title      36
author    132
text        3
label       0
dtype: int64

In [29]:
# filling null values with empty string
news_dataset=news_dataset.fillna('')

In [30]:
#we are merging the title and author together for prediction becoz text take too much time for processing becoz it is too large
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [31]:
news_dataset['content']

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
1252    EdJenner Michael Moore: Forget What I Said Bef...
1253                            Why Standing Rock Matters
1254     Muslims in Kashmir observe martyrdom anniv. o...
1255    shorty The Lethal Lie of Hillary Clinton: Savi...
1256    Sara Vallone When Dad’s Kind Birthday Gift For...
Name: content, Length: 1257, dtype: object

In [32]:
news_dataset

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
1252,1252,Michael Moore: Forget What I Said Before. Trum...,EdJenner,Go to Article \nFilmmaker Michael Moore made a...,1,EdJenner Michael Moore: Forget What I Said Bef...
1253,1253,Why Standing Rock Matters,,Right to Clean Water & The Fight Against Emine...,1,Why Standing Rock Matters
1254,1254,Muslims in Kashmir observe martyrdom anniv. of...,,Muslims in Kashmir observe martyrdom anniv. of...,1,Muslims in Kashmir observe martyrdom anniv. o...
1255,1255,The Lethal Lie of Hillary Clinton: Saving Live...,shorty,ABOVE: Image by DonkeyHotey | CC BY 2.0 “We mu...,1,shorty The Lethal Lie of Hillary Clinton: Savi...


In [33]:
#seperating data and label
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']


In [34]:
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
1252,1252,Michael Moore: Forget What I Said Before. Trum...,EdJenner,Go to Article \nFilmmaker Michael Moore made a...,EdJenner Michael Moore: Forget What I Said Bef...
1253,1253,Why Standing Rock Matters,,Right to Clean Water & The Fight Against Emine...,Why Standing Rock Matters
1254,1254,Muslims in Kashmir observe martyrdom anniv. of...,,Muslims in Kashmir observe martyrdom anniv. of...,Muslims in Kashmir observe martyrdom anniv. o...
1255,1255,The Lethal Lie of Hillary Clinton: Saving Live...,shorty,ABOVE: Image by DonkeyHotey | CC BY 2.0 “We mu...,shorty The Lethal Lie of Hillary Clinton: Savi...


In [35]:
Y

0       1
1       0
2       1
3       1
4       1
       ..
1252    1
1253    1
1254    1
1255    1
1256    1
Name: label, Length: 1257, dtype: int64

steamming is the process of reducing a word to its root word 

example : acting , actress , acotor -> act(rootword) because we need to reduce word as much as possible to have a better performance on our model 

once we reduce the word to its root word we perform the function called vectorization we convert the text column into its corresponding feature vector

In [37]:
port_stem=PorterStemmer()

In [40]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # remove all the conetnt for the word except a-z adnd A-Z if there is numbers commas or special characters its replace by the space thats why we mentioned space there 
    stemmed_content = stemmed_content.lower() # all uppercase letters are converted into lowercase letter becoz it cause problem in processing 
    stemmed_content = stemmed_content.split() # all the words are converted in lowercase letter and converted into the list 
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [41]:
news_dataset['content'] = news_dataset['content'].apply(stemming) # stemming function get applied on the column called content 

In [42]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
1252    edjenn michael moor forget said trump support ...
1253                                    stand rock matter
1254    muslim kashmir observ martyrdom anniv th shia ...
1255    shorti lethal lie hillari clinton save live fl...
1256    sara vallon dad kind birthday gift ex wife fri...
Name: content, Length: 1257, dtype: object


In [43]:
# we are only using title and author for processing as combine called content 
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [44]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'muslim kashmir observ martyrdom anniv th shia imam'
 'shorti lethal lie hillari clinton save live fli zone syria'
 'sara vallon dad kind birthday gift ex wife friend puzzl explain realli']


In [45]:
print(Y)

[1 0 1 ... 1 1 1]


In [46]:
Y.shape

(1257,)

In [47]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [48]:
print(X)

  (0, 4028)	0.3024183757846162
  (0, 3427)	0.27030661635553566
  (0, 2300)	0.32608363765199083
  (0, 2225)	0.3024183757846162
  (0, 1996)	0.254718706861023
  (0, 1826)	0.23819485692645506
  (0, 1299)	0.25815194216850107
  (0, 1015)	0.28064429389202084
  (0, 961)	0.32608363765199083
  (0, 771)	0.25815194216850107
  (0, 634)	0.344867812750182
  (0, 79)	0.28064429389202084
  (1, 4271)	0.3064856641632665
  (1, 1785)	0.20053214950737855
  (1, 1457)	0.677707181991322
  (1, 954)	0.28647686401541284
  (1, 729)	0.211620185764401
  (1, 564)	0.3975199855021747
  (1, 492)	0.16660934038468617
  (1, 389)	0.3064856641632665
  (2, 4014)	0.4487249337789125
  (2, 2466)	0.46406386169938657
  (2, 1574)	0.3569567199284081
  (2, 1421)	0.3687697375513159
  (2, 818)	0.46406386169938657
  :	:
  (1254, 2074)	0.3717171395125874
  (1254, 1876)	0.3717171395125874
  (1254, 162)	0.3717171395125874
  (1255, 4339)	0.3381619892045228
  (1255, 3821)	0.269004874443607
  (1255, 3524)	0.349721503703015
  (1255, 3371)	0.385


Splitting the dataset to training & test data

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [50]:
model = LogisticRegression()

In [51]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [52]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [53]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9751243781094527


In [54]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [55]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9087301587301587


Making a Predictive System

In [56]:
X_new = X_test[3] # X_new store fourth row for X_test dataset
prediction = model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [57]:
print(Y_test[3])

0
